In [73]:
import sqlite3
import pandas as pd
import csv
from tabulate import tabulate

### Checking out the columns

In [50]:
company=pd.read_csv("/Users/lohitashwa/Desktop/Task/clean_nasdaq_screener.csv")
company.columns

Index(['Symbol', 'Last Sale', 'Market Cap', 'Country', 'Sector', 'Industry'], dtype='object')

In [6]:
balance_s=pd.read_csv("/Users/lohitashwa/Desktop/Task/clean_balance_sheet.csv")
balance_s.columns

Index(['Date', 'Company Symbol', 'Accounts Payable', 'Capital Surplus',
       'Cash and Cash Equivalents', 'Common Stocks', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Fixed Assets', 'Goodwill',
       'Intangible Assets', 'Inventory', 'Long-Term Debt',
       'Long-Term Investments', 'Minority Interest', 'Misc. Stocks',
       'Net Receivables', 'Other Assets', 'Other Current Assets',
       'Other Current Liabilities', 'Other Equity', 'Other Liabilities',
       'Retained Earnings',
       'Short-Term Debt / Current Portion of Long-Term Debt',
       'Short-Term Investments', 'Total Assets', 'Total Current Assets',
       'Total Current Liabilities', 'Total Equity', 'Total Liabilities',
       'Total Liabilities & Equity', 'Treasury Stock'],
      dtype='object')

In [4]:
income_s=pd.read_csv("/Users/lohitashwa/Desktop/Task/clean_income_statement.csv")
income_s.columns

Index(['Date', 'Company Symbol', 'Add'l income/expense items',
       'Cost of Revenue', 'Earnings Before Interest and Tax',
       'Earnings Before Tax', 'Equity Earnings/Loss Unconsolidated Subsidiary',
       'Gross Profit', 'Income Tax', 'Interest Expense', 'Minority Interest',
       'Net Income', 'Net Income Applicable to Common Shareholders',
       'Net Income-Cont. Operations', 'Non-Recurring Items',
       'Operating Income', 'Other Operating Items', 'Research and Development',
       'Sales, General and Admin.', 'Total Revenue'],
      dtype='object')

# SQL

### connecting financial database financial 

In [74]:
# Connect to an SQLite database
connection = sqlite3.connect('/Users/lohitashwa/Desktop/Task/finance_data.db')

In [75]:
cursor = connection.cursor()

### Create DB such as defining the structure of the table and defining relationship

In [76]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS company (
        Company_Symbol TEXT PRIMARY KEY,
        Last_Sale REAL,
        Market_Cap REAL,
        Country TEXT,
        Sector TEXT,
        Industry TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS balance_sheet (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Date TEXT,
        Company_Symbol TEXT,
        Accounts_Payable REAL,
        Capital_Surplus REAL,
        Cash_and_Cash_Equivalents REAL,
        Common_Stocks REAL,
        Deferred_Asset_Charges REAL,
        Deferred_Liability_Charges REAL,
        Fixed_Assets REAL,
        Goodwill REAL,
        Intangible_Assets REAL,
        Inventory REAL,
        Long_Term_Debt REAL,
        Long_Term_Investments REAL,
        Minority_Interest REAL,
        Misc_Stocks REAL,
        Net_Receivables REAL,
        Other_Assets REAL,
        Other_Current_Assets REAL,
        Other_Current_Liabilities REAL,
        Other_Equity REAL,
        Other_Liabilities REAL,
        Retained_Earnings REAL,
        Short_Term_Debt_Current_Portion_of_Long_Term_Debt REAL,
        Short_Term_Investments REAL,
        Total_Assets REAL,
        Total_Current_Assets REAL,
        Total_Current_Liabilities REAL,
        Total_Equity REAL,
        Total_Liabilities REAL,
        Total_Liabilities_Equity REAL,
        Treasury_Stock REAL,
        FOREIGN KEY (Company_Symbol) REFERENCES company(Company_Symbol)
    )
''')


cursor.execute('''
    CREATE TABLE IF NOT EXISTS income_statement (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Date TEXT,
        Company_Symbol TEXT,
        Addl_income_expense_items REAL,
        Cost_of_Revenue REAL,
        Earnings_Before_Interest_and_Tax REAL,
        Earnings_Before_Tax REAL,
        Equity_Earnings_Loss_Unconsolidated_Sub REAL,
        Gross_Profit REAL,
        Income_Tax REAL,
        Interest_Expense REAL,
        Minority_Interest REAL,
        Net_Income REAL,
        Net_Income_Applicable_to_Common_Shareholders REAL,
        Net_Income_Cont_Operations REAL,
        Non_Recurring_Items REAL,
        Operating_Income REAL,
        Other_Operating_Items REAL,
        Research_and_Development REAL,
        Sales_General_and_Admin REAL,
        Total_Revenue REAL,
        FOREIGN KEY (Company_Symbol) REFERENCES company(Company_Symbol)
    )
''')


### Generalized Function (insert_data_into_table) for inserting data into the table with respect to their csv 

In [77]:
def insert_data_into_table(file_path, table_name, column_names, cursor):
    connection = cursor.connection

    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        # Skip the header row if it exists
        next(csv_reader, None)
        for row in csv_reader:
            values = []
            for i in range(len(column_names)):
                if i == 1 or i == 2:
                    values.append(float(row[i]) if row[i].replace('.', '', 1).isdigit() else row[i])
                else:
                    values.append(row[i])
            placeholders = ', '.join(['?'] * len(column_names))
            sql = "INSERT INTO {} ({}) VALUES ({})".format(table_name, ', '.join(column_names), placeholders)
            cursor.execute(sql, values)

In [78]:
# 'company' table
insert_data_into_table('/Users/lohitashwa/Desktop/Task/clean_nasdaq_screener.csv',
                       'company',
                       ['Company_Symbol', 'Last_Sale', 'Market_Cap', 'Country', 'Sector', 'Industry'],cursor)

#'balance_sheet' table
insert_data_into_table('/Users/lohitashwa/Desktop/Task/clean_balance_sheet.csv',
                       'balance_sheet',
                       ['Date', 'Company_Symbol', 'Accounts_Payable', 'Capital_Surplus', 'Cash_and_Cash_Equivalents', 'Common_Stocks', 'Deferred_Asset_Charges', 'Deferred_Liability_Charges', 'Fixed_Assets', 'Goodwill', 'Intangible_Assets', 'Inventory', 'Long_Term_Debt', 'Long_Term_Investments', 'Minority_Interest', 'Misc_Stocks', 'Net_Receivables', 'Other_Assets', 'Other_Current_Assets', 'Other_Current_Liabilities', 'Other_Equity', 'Other_Liabilities', 'Retained_Earnings', 'Short_Term_Debt_Current_Portion_of_Long_Term_Debt', 'Short_Term_Investments', 'Total_Assets', 'Total_Current_Assets', 'Total_Current_Liabilities', 'Total_Equity', 'Total_Liabilities', 'Total_Liabilities_Equity', 'Treasury_Stock'],cursor)


# 'income_statement' table
insert_data_into_table('/Users/lohitashwa/Desktop/Task/clean_income_statement.csv',
                       'income_statement',
                       ['Date', 'Company_Symbol', 'Addl_income_expense_items', 'Cost_of_Revenue', 'Earnings_Before_Interest_and_Tax', 'Earnings_Before_Tax', 'Equity_Earnings_Loss_Unconsolidated_Sub', 'Gross_Profit', 'Income_Tax', 'Interest_Expense', 'Minority_Interest', 'Net_Income', 'Net_Income_Applicable_to_Common_Shareholders', 'Net_Income_Cont_Operations', 'Non_Recurring_Items', 'Operating_Income', 'Other_Operating_Items', 'Research_and_Development', 'Sales_General_and_Admin', 'Total_Revenue'],cursor)


In [83]:
connection.commit()

### Display Some Selected Queries to check that data has Been Inserted Correctly

In [79]:

cursor.execute("SELECT * FROM company")

# Fetch the first 5 rows
rows = cursor.fetchmany(5)

column_names = [description[0] for description in cursor.description]

# results in a tabular format
print(tabulate(rows, headers=column_names, tablefmt="grid"))

+------------------+-------------+--------------+---------------+------------------------+----------------------+
| Company_Symbol   |   Last_Sale |   Market_Cap | Country       | Sector                 | Industry             |
+==================+=============+==============+===============+========================+======================+
| A                |      118.73 |  3.47389e+10 | United States | Industrials            | Electrical Products  |
+------------------+-------------+--------------+---------------+------------------------+----------------------+
| AA               |       30.29 |  5.40524e+09 | United States | Industrials            | Aluminum             |
+------------------+-------------+--------------+---------------+------------------------+----------------------+
| AAC              |       10.7  |  7.55466e+08 | United States | Industrials            | Metal Fabrications   |
+------------------+-------------+--------------+---------------+-----------------------

In [80]:
cursor.execute("SELECT * FROM balance_sheet")

rows = cursor.fetchmany(5)

for row in rows:
    print(row)

(1, '10/31/2022', 'A', 1364000.0, 324000.0, 1053000.0, 3000.0, 500654.57736081205, 545758.0158412796, 1100000.0, 3952000.0, 821000.0, 1038000.0, 2733000.0, 195000.0, 210414.1210582987, 539508.0319901569, 1405000.0, 686000.0, 282000.0, 461000.0, -347000.0, 633000.0, -2587713.1149230767, 36000.0, 981105.8143472907, 10532000.0, 3778000.0, 1861000.0, 5305000.0, 5227000.0, 10532000.0, 5325000.0)
(2, '10/31/2022', 'AA', 2322000.0, -570000.0, 1631000.0, 2000.0, 296000.0, 215000.0, 6493000.0, 3952000.0, 821000.0, 2427000.0, 2832000.0, 1124000.0, 210414.1210582987, 1513000.0, 909000.0, 1593000.0, 417000.0, 481000.0, -3539000.0, 2116000.0, -2587713.1149230767, 201000.0, 981105.8143472907, 14756000.0, 5250000.0, 3004000.0, 5076000.0, 9680000.0, 14756000.0, 9183000.0)
(3, '10/31/2022', 'AAC', 8176.0, -60959.0, 38.0, 3.0, 296000.0, 35000.0, 6493000.0, 3952000.0, 821000.0, 2427000.0, 2832000.0, 1013382.0, 1013282.0, 1513000.0, 909000.0, 1593000.0, 58.0, 481000.0, -3539000.0, 16476.0, -2587713.114923

In [81]:
cursor.execute("SELECT * FROM income_statement")


rows = cursor.fetchmany(5)

# Display the results
for row in rows:
    print(row)


(1, '10/31/2022', 'A', -30000.0, 3126000.0, 1588000.0, 1504000.0, -21220.16151866152, 3722000.0, 250000.0, 84000.0, 86265.93497363797, 1254000.0, 1254000.0, 1254000.0, 221314.0407643312, 1618000.0, 508059.0050880626, 467000.0, 1637000.0, 6848000.0)
(2, '10/31/2022', 'AA', 118000.0, 10212000.0, 808000.0, 702000.0, -161000.0, 2239000.0, 664000.0, 106000.0, 86265.93497363797, -123000.0, -123000.0, -123000.0, 696000.0, 690000.0, 617000.0, 32000.0, 204000.0, 12451000.0)
(3, '10/31/2022', 'AAC', 28326.0, 3588572.0505489954, 20984.0, 20984.0, -21220.16151866152, 2163320.9701934466, 169064.9140401146, 165196.74118165785, 86265.93497363797, 20984.0, 20984.0, 20984.0, 221314.0407643312, -7343.0, 508059.0050880626, 309438.0238290879, 7343.0, 5448493.066715143)
(4, '10/31/2022', 'AACG', 189.0, 15006.0, -7842.0, -7842.0, 101.0, 14745.0, -852.0, 165196.74118165785, 86265.93497363797, -6889.0, -6889.0, -6889.0, 221314.0407643312, -8140.0, 508059.0050880626, 977.0, 21908.0, 29751.0)
(5, '10/31/2022', 

In [82]:

cursor.execute("SELECT Company_Symbol, Date FROM balance_sheet")


rows = cursor.fetchmany(5)


column_names = [description[0] for description in cursor.description]


table = tabulate(rows, headers=column_names, tablefmt="grid")

print(table)


+------------------+------------+
| Company_Symbol   | Date       |
+==================+============+
| A                | 10/31/2022 |
+------------------+------------+
| AA               | 10/31/2022 |
+------------------+------------+
| AAC              | 10/31/2022 |
+------------------+------------+
| AACG             | 10/31/2022 |
+------------------+------------+
| AACI             | 10/31/2022 |
+------------------+------------+


In [84]:
cursor.execute("SELECT Date, Company_Symbol, Addl_income_expense_items FROM income_statement")

rows = cursor.fetchmany(5)


column_names = [description[0] for description in cursor.description]


table = tabulate(rows, headers=column_names, tablefmt="grid")

print(table)


+------------+------------------+-----------------------------+
| Date       | Company_Symbol   |   Addl_income_expense_items |
+============+==================+=============================+
| 10/31/2022 | A                |                      -30000 |
+------------+------------------+-----------------------------+
| 10/31/2022 | AA               |                      118000 |
+------------+------------------+-----------------------------+
| 10/31/2022 | AAC              |                       28326 |
+------------+------------------+-----------------------------+
| 10/31/2022 | AACG             |                         189 |
+------------+------------------+-----------------------------+
| 10/31/2022 | AACI             |                        1026 |
+------------+------------------+-----------------------------+


### Aggregate Data:
#### • Use GROUP BY to group data by one or more columns.
#### • Apply aggregate functions like SUM, AVG, COUNT, etc., to calculate summary statistics.
#### • Employ the HAVING clause to filter grouped data based on aggregate conditions.

In [85]:
def display(cursor):
    rows = cursor.fetchmany(5)


    column_names = [description[0] for description in cursor.description]

    table = tabulate(rows, headers=column_names, tablefmt="grid")

    print(table)

####  Calculate the average 'Market_Cap' for companies in each sector, including data from the 'company' and 'balance_sheet' tables using JOIN

In [96]:
display(cursor.execute("SELECT c.Sector, AVG(c.Market_Cap) AS Average_Market_Cap FROM company c JOIN balance_sheet b ON c.Company_Symbol = b.Company_Symbol GROUP BY c.Sector order BY c.Sector DESC"))


+--------------------+----------------------+
| Sector             |   Average_Market_Cap |
+====================+======================+
| Utilities          |          1.16959e+10 |
+--------------------+----------------------+
| Telecommunications |          1.78864e+10 |
+--------------------+----------------------+
| Technology         |          2.32262e+10 |
+--------------------+----------------------+
| Real Estate        |          5.47569e+09 |
+--------------------+----------------------+
| Miscellaneous      |          1.8724e+09  |
+--------------------+----------------------+


#### Calculate the total 'Total_Assets' for each country on different dates, including data from the 'balance_sheet' and 'company' tables using JOIN.

In [97]:
display(cursor.execute("SELECT c.Country, b.Date, SUM(b.Total_Assets) AS Total_Assets FROM company c JOIN balance_sheet b ON c.Company_Symbol = b.Company_Symbol GROUP BY c.Country, b.Date"))

+-----------+------------+----------------+
| Country   | Date       |   Total_Assets |
+===========+============+================+
| Argentina | 10/31/2019 |    1.38344e+08 |
+-----------+------------+----------------+
| Argentina | 10/31/2020 |    1.18219e+08 |
+-----------+------------+----------------+
| Argentina | 10/31/2021 |    1.6673e+08  |
+-----------+------------+----------------+
| Argentina | 10/31/2022 |    1.17255e+08 |
+-----------+------------+----------------+
| Australia | 10/31/2019 |    3.08747e+08 |
+-----------+------------+----------------+


#### groups the data by 'Country' and calculates the average 'Market_Cap' for companies in each country, providing insights into the distribution of market capitalization

In [87]:
display(cursor.execute("SELECT Country, AVG(Market_Cap) AS Average_Market_Cap FROM company GROUP BY Country"))


+-----------+----------------------+
| Country   |   Average_Market_Cap |
+===========+======================+
| Argentina |          5.88885e+09 |
+-----------+----------------------+
| Australia |          2.17937e+10 |
+-----------+----------------------+
| Bahamas   |          1.1449e+09  |
+-----------+----------------------+
| Belgium   |          1.99807e+10 |
+-----------+----------------------+
| Bermuda   |          5.95644e+09 |
+-----------+----------------------+


#### groups the data by 'Sector' and calculates the total 'Net_Income' for companies in each sector, offering insights into the profitability of different business sectors.

In [88]:
display(cursor.execute("SELECT Sector, SUM(Net_Income) AS Total_Net_Income FROM income_statement JOIN company ON income_statement.Company_Symbol = company.Company_Symbol GROUP BY Sector "))

+------------------------+--------------------+
| Sector                 |   Total_Net_Income |
+========================+====================+
| Basic Materials        |        2.40327e+08 |
+------------------------+--------------------+
| Consumer Discretionary |        1.26278e+09 |
+------------------------+--------------------+
| Consumer Staples       |        3.23335e+08 |
+------------------------+--------------------+
| Energy                 |        5.03302e+08 |
+------------------------+--------------------+
| Finance                |        2.59837e+09 |
+------------------------+--------------------+


#### combines data from both the 'balance_sheet' and 'income_statement' tables, groups it by 'Date,' and counts the number of unique companies reporting financial data on each date, giving insights into data availability over time.

In [89]:
display(cursor.execute("SELECT Date, COUNT(DISTINCT Company_Symbol) AS Company_Count FROM (SELECT Date, Company_Symbol FROM balance_sheet UNION SELECT Date, Company_Symbol FROM income_statement) AS combined_data GROUP BY Date"))

+------------+-----------------+
| Date       |   Company_Count |
+============+=================+
| 10/31/2019 |            6503 |
+------------+-----------------+
| 10/31/2020 |            6503 |
+------------+-----------------+
| 10/31/2021 |            6503 |
+------------+-----------------+
| 10/31/2022 |            6503 |
+------------+-----------------+


#### Select Data:
#### • Use SELECT statements to retrieve specific columns and rows.
#### • Apply WHERE clauses to filter data based on specified conditions.
#### • Use ORDER BY to sort data.
#### • Utilize TOP to limit the number of rows returned (this depends on the database system being used).

#### This query utilizes a subquery to find the companies with the highest net income, and then it joins the result with the 'company' table to filter for companies in the Technology sector. This provides insights into the most profitable technology companies and the dates on which they achieved their highest net income.

In [90]:
display(cursor.execute("SELECT c.Company_Symbol, i.Date, i.Net_Income FROM income_statement i JOIN (SELECT Company_Symbol, MAX(Net_Income) AS Max_Net_Income FROM income_statement GROUP BY Company_Symbol )AS max_income ON i.Company_Symbol = max_income.Company_Symbol AND i.Net_Income = max_income.Max_Net_Income JOIN company c ON i.Company_Symbol = c.Company_Symbol WHERE c.Sector = 'Technology'"))

+------------------+------------+------------------+
| Company_Symbol   | Date       |       Net_Income |
+==================+============+==================+
| AAPL             | 10/31/2022 |      9.9803e+07  |
+------------------+------------+------------------+
| ACIW             | 10/31/2022 | 142177           |
+------------------+------------+------------------+
| ACLS             | 10/31/2022 | 183079           |
+------------------+------------+------------------+
| ACMR             | 10/31/2022 |  39263           |
+------------------+------------+------------------+
| ADI              | 10/31/2022 |      2.74856e+06 |
+------------------+------------+------------------+


#### This query uses a subquery to find the latest date for each company in the 'balance_sheet' table. It then calculates the ratio of 'Total_Assets' to 'Market_Cap' and identifies the companies with the highest ratios. This query provides insights into companies with strong asset backing relative to their market capitalization.

In [91]:
display(cursor.execute("SELECT c.Company_Symbol, (b.Total_Assets / c.Market_Cap) AS Asset_to_MarketCap_Ratio FROM company c JOIN balance_sheet b ON c.Company_Symbol = b.Company_Symbol WHERE b.Date = (SELECT MAX(Date) FROM balance_sheet WHERE Company_Symbol = c.Company_Symbol)ORDER BY Asset_to_MarketCap_Ratio DESC LIMIT 10 "))

+------------------+----------------------------+
| Company_Symbol   |   Asset_to_MarketCap_Ratio |
+==================+============================+
| CTBB             |                1.57708e+06 |
+------------------+----------------------------+
| QVCC             |                1.12764e+06 |
+------------------+----------------------------+
| VIEWW            |               14.3141      |
+------------------+----------------------------+
| HCDIZ            |               14.2802      |
+------------------+----------------------------+
| HCDIW            |                8.44898     |
+------------------+----------------------------+


#### This query applies a WHERE clause to filter companies with 'Total_Assets' within the specified range, offering insights into companies with moderate asset levels.

In [92]:
display(cursor.execute("SELECT Date, Total_Assets FROM balance_sheet WHERE Total_Assets BETWEEN 1000000000 AND 10000000000 "))

+------------+----------------+
| Date       |   Total_Assets |
+============+================+
| 10/31/2022 |    3.05138e+09 |
+------------+----------------+
| 10/31/2022 |    1.82084e+09 |
+------------+----------------+
| 10/31/2022 |    2.41668e+09 |
+------------+----------------+
| 10/31/2022 |    1.43463e+09 |
+------------+----------------+
| 10/31/2022 |    1.4418e+09  |
+------------+----------------+
